# Structured RAG using Amazon Bedrock Knowledge Bases - End to end example using Amazon Redshift DB

Structure RAG allows Amazon Bedrock Knowledge Bases customers to query structured data in Redshift using natural language, and receive natural language responses summarizing the data thereby providing an answer to the user question.

Using advanced natural language processing, Bedrock Knowledge Bases can transform natural language queries into SQL queries, allowing users to retrieve data directly from the source without the need to move or preprocess the data. o generate accurate SQL queries, Bedrock Knowledge Base leverages database schema, previous query history, and other contextual information that are provided about the data sources.


This notebook provides sample code for building a Structured RAG using Amazon Bedrock Knowledgebases using Redshift.


#### Steps: 
- Create Knowledge Base execution role with necessary policies for accessing data from Amazon Redshift.
- Create a knowledge base with Structured database (Redshift database)
- Create data source(s) within knowledge base
- Start ingestion jobs using KB APIs which will read metadata about structred database and ingest into QDBC.

Once the metadata is extracted and ingested, then user can interact with Structured databases via Amazon Bedrock Knowledge Base APIs using Natural language query.




<div class="alert alert-block alert-warning">
<b>Note:</b> NOTE: There're multiple access pattern for Redshift provisioned cluster (i.e. DB user + Cluster, IAM + Cluster, and Secrets + Cluster.) and WorkGroup (i.e.  Secret + Workgroup, IAM + Workgroup). 

In this notebook, we'll focus on,  Secret + Workgroup access pattern (Redshift Serverless).
</div>

## Setup
Before running the rest of this notebook, you'll need to run the cells below to (ensure necessary libraries are installed and) connect to Bedrock.

Please ignore any pip dependency error (if you see any while installing libraries)

In [ ]:
%pip install --force-reinstall -q -r ../requirements.txt --quiet

# %pip install boto3 opensearch-py botocore awscli retrying ragas=0.1.9 ipywidgets  iprogressretrying langchain langchain_aws langchain_community --quiet 

In [ ]:
%pip install --upgrade boto3
import boto3
print(boto3.__version__)

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

#### Pre-requisites
This notebook requires :
- A Redshift cluster or workgroup already setup with your structured data ingested into the cluster
- You've set-up the secrets manager for Workgroup & Secrets access pattern


In [20]:
import json
import os
import boto3
from botocore.exceptions import ClientError
import pprint
from utility import create_bedrock_execution_role_structured_rag, create_bedrock_execution_role_multi_ds, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss, interactive_sleep

import random
from retrying import retry
suffix = random.randrange(200, 900)

sts_client = boto3.client('sts')
boto3_session = boto3.session.Session(region_name='us-east-1')
region_name = boto3_session.region_name
account_id = sts_client.get_caller_identity()["Account"]

bedrock_agent_client = boto3.client('bedrock-agent', region_name=region_name)
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name=region_name)


pp = pprint.PrettyPrinter(indent=2)

### Provide Structured data store details

Currently Amazon Bedrock Knowledge Bases supports Amazon Redshift (A fully managed, petabyte-scale data warehouse service in the cloud that lets you access and analyze data without needing to configure provisioning and capacity.) as data store.

#### Set-up few variables and configurations needed for Structured RAG

In [2]:
# Change below variables as needed

name = f"bedrock-sample-structured-kb-{suffix}"
description = "Sample Structured KB."

redshiftDBName = "sample_data_dev"
workgroupArn =  "arn:aws:redshift-serverless:us-east-1:017444429555:workgroup/ad3a0804-6ecf-4636-a798-a00c7f1293a7"
usernamePasswordSecretArn = "arn:aws:secretsmanager:us-east-1:017444429555:secret:redshift-riv-test-user-h86sJH"

Redshift acces pattern choice

In [3]:
import ipywidgets as widgets
from IPython.display import display

# Display the choices to the user
print("Choose your Redshift access pattern:")
print("1. SECRET+WG")
print("2. IAM+WG")

# Create a text input widget
choice_widget = widgets.Text(placeholder="Enter your choice (1-2)")

# Display the widget
display(choice_widget)


Choose your Redshift access pattern:
1. SECRET+WG
2. IAM+WG


Text(value='', placeholder='Enter your choice (1-2)')

In [4]:
# Access the entered value
access_pattern_choice = int(choice_widget.value)
try:
    access_pattern_choice = int(choice_widget.value)
    if access_pattern_choice ==1:
        print(f"Access pattern:{choice_widget.value} SECRET+WG")
        workgroup_Arn = workgroupArn
        usernamePasswordSecret_Arn = usernamePasswordSecretArn
        
        bedrock_kb_execution_role = create_bedrock_execution_role_structured_rag(workgroup_arn = workgroupArn, secrets_arn = usernamePasswordSecret_Arn)
        bedrock_kb_execution_role_arn = bedrock_kb_execution_role['Role']['Arn']
    else:
        print("Invalid choice. Please enter a number between 1 and 2.")
except ValueError:
    print("Invalid input. Please enter a number.")

Access pattern:1 SECRET+WG


In [5]:
bedrock_kb_execution_role_arn

'arn:aws:iam::017444429555:role/AmazonBedrockExecutionRoleForKnowledgeBase_274'

### Define Knowledge Bases Configuration to set-up Structured RAG

In [6]:
kbConfigParam = {
            "type": "SQL",
            "sqlKnowledgeBaseConfiguration": {
                "type": "REDSHIFT",
                "redshiftConfiguration": {
                    "storageConfigurations": [{
                        "type": "REDSHIFT",
                        "redshiftConfiguration": {
                            "databaseName": redshiftDBName
                        }
                    }],
                    "queryEngineConfiguration": {
                        "type": "SERVERLESS",
                        "serverlessConfiguration": {
                            "workgroupArn": workgroupArn,
                            "authConfiguration": {
                                "type": "USERNAME_PASSWORD",
                                "usernamePasswordSecretArn": usernamePasswordSecretArn
                            }
                        }
                    }
                }
            }
        }

## Create Knowledge Bases

In [7]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_structured_kb_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = name,
        description = description,
        roleArn = bedrock_kb_execution_role_arn,
        knowledgeBaseConfiguration = kbConfigParam
    )
    return create_kb_response["knowledgeBase"]

In [8]:
try:
    kb = create_structured_kb_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

In [9]:
# Print the knowledge base Id in bedrock, that corresponds to the Opensearch index in the collection we created before, we will use it for the invocation later
kb_id = kb["knowledgeBaseId"]
pp.pprint(kb_id)

'O3AWVPVFSA'


## Create Data Source(s)

In [11]:
## Create Data Source

response = bedrock_agent_client.create_data_source(
            dataSourceConfiguration= {
                "type": "REDSHIFT_METADATA"
            },
            name="snghigf-redshift-DS",
            description="Metadata data source for redshift" ,
            knowledgeBaseId=kb['knowledgeBaseId']
        )
            

In [12]:
data_source_id = response['dataSource']['dataSourceId']
print(f"Data source created successfully. ID: {data_source_id}")

Data source created successfully. ID: CDVVQZHXWP


## Start Ingestion job

In [13]:
import time
try:
    start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = data_source_id)
    job = start_job_response["ingestionJob"]
    print(f"job started successfully\n")

    while job['status'] not in ["COMPLETE", "FAILED", "STOPPED"]:
        get_job_response = bedrock_agent_client.get_ingestion_job(
            knowledgeBaseId = kb['knowledgeBaseId'],
            dataSourceId = data_source_id,
            ingestionJobId = job["ingestionJobId"]
        )
        job = get_job_response["ingestionJob"]
    pp.pprint(job)
    time.sleep(10)

except Exception as e:
    print(f"Couldn't start ingest job.\n")
    print(e)
    

job started successfully

{ 'dataSourceId': 'CDVVQZHXWP',
  'ingestionJobId': '8JPTS948Y6',
  'knowledgeBaseId': 'O3AWVPVFSA',
  'startedAt': datetime.datetime(2024, 12, 10, 2, 12, 52, 139604, tzinfo=tzutc()),
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2024, 12, 10, 2, 13, 49, 690763, tzinfo=tzutc())}


In [14]:
# keep the kb_id for invocation later in the invoke request
%store kb_id

Stored 'kb_id' (str)


## Test the Structured RAG

In [15]:
query = "What are the venues are there in new york city?"

### Using RetrieveAndGenerate API

In [21]:
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region_name, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

The query returned a list of venues located in New York City, including Carnegie Hall, Yankee Stadium, Booth Theatre, Music Box Theatre, Ambassador Theatre, Hilton Theatre, Al Hirschfeld Theatre, Marquis Theatre, Studio 54, and Richard Rodgers Theatre among others. In total, there are 45 venues listed.



### Using Retrieve API

In [22]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        "text": query
    }
)

In [23]:
import json
import pandas as pd

#Function to extract retrieved results from Retrieve API response

def response_print(retrieve_resp):

    # Extract the retrievalResults list
    retrieval_results = retrieve_resp['retrievalResults']

    # Dictionary to store the extracted data
    extracted_data = {}

    # Iterate through each item in retrievalResults
    for item in retrieval_results:
        row = item['content']['row']
        for col in row:
            column_name = col['columnName']
            column_value = col['columnValue']
            
            # If this column hasn't been seen before, create a new list for it
            if column_name not in extracted_data:
                extracted_data[column_name] = []
            
            # Append the value to the appropriate list
            extracted_data[column_name].append(column_value)

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(extracted_data)

    return df
    

In [24]:
# Display the Retrieved results records
df = response_print(response_ret)
print(df.head())

            venuename      venuecity venuestate
0       Carnegie Hall  New York City         NY
1      Yankee Stadium  New York City         NY
2       Booth Theatre  New York City         NY
3   Music Box Theatre  New York City         NY
4  Ambassador Theatre  New York City         NY


### Using Generate Query

In [30]:
query_response = bedrock_agent_runtime_client.generate_query(
    queryGenerationInput={
        "text": query,
        "type": "TEXT"
    },
    transformationConfiguration={
        "mode" : "TEXT_TO_SQL",
        "textToSqlConfiguration": {
            "type": "KNOWLEDGE_BASE",
            "knowledgeBaseConfiguration": {
                "knowledgeBaseArn": kb['knowledgeBaseArn']
            }
        }
    }
)

generated_sql = query_response['queries'][0]['sql']
generated_sql

"SELECT v.venuename, v.venuecity, v.venuestate FROM tickit.venue v WHERE v.venuecity = 'New York City';"

## Clean Up
Please make sure to uncomment and run the below section to delete all the resources

In [31]:
bedrock_agent_client.delete_data_source(dataSourceId = data_source_id, knowledgeBaseId=kb['knowledgeBaseId'])
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb['knowledgeBaseId'])

{'ResponseMetadata': {'RequestId': 'e97a0e58-cfcf-4d5a-ba13-376b1eefc60b',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 02:25:57 GMT',
   'content-type': 'application/json',
   'content-length': '52',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e97a0e58-cfcf-4d5a-ba13-376b1eefc60b',
   'x-amz-apigw-id': 'Cjb8XHdsoAMEGuw=',
   'x-amzn-trace-id': 'Root=1-6757a6b5-07df9c5f3fa94c9d701939b7'},
  'RetryAttempts': 0},
 'knowledgeBaseId': 'O3AWVPVFSA',
 'status': 'DELETING'}

In [32]:
# Delete role and policies

iam = boto3.resource('iam')
client = boto3.client('iam')

role_name = bedrock_kb_execution_role['Role']['RoleName']

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

attached_policies = get_attached_policies(role_name)
attached_policies

for policy in attached_policies:
        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        client.detach_role_policy(RoleName=role_name,
            PolicyArn=policy_arn)
        
        response = client.delete_policy(
            PolicyArn=policy_arn
        )
        print(response)

client.delete_role(RoleName=role_name)

arn:aws:iam::017444429555:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_274
{'ResponseMetadata': {'RequestId': 'a2d2b9a6-dbdc-4476-9a3e-236bade5fe5e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 02:26:01 GMT', 'x-amzn-requestid': 'a2d2b9a6-dbdc-4476-9a3e-236bade5fe5e', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}


{'ResponseMetadata': {'RequestId': 'e3f2fee3-faf6-43f3-bfd9-d42da088ede4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 10 Dec 2024 02:26:02 GMT',
   'x-amzn-requestid': 'e3f2fee3-faf6-43f3-bfd9-d42da088ede4',
   'content-type': 'text/xml',
   'content-length': '200'},
  'RetryAttempts': 0}}